## Overview

pandas is a very useful tool for manipulating data, but it's heavy on memory resource due to the amount of features it provides. People (ab)use pandas out of convenience for opening big CSV, Excel & other file formats, which is fine for most part, but sometimes using pandas just to take a look & manipulating a little bit of big data file is slow & cumbersome. If this is relevant to you but you need to use pandas then you need to use `chunksize` parameter for [`pd.read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

I'm going to use python generator to demonstrate how to achieve this kind of task with built-in modules, although I suspect it _might be_ more expensive on I/O side, but speed wise this is should be faster because this one only lazily loads the data when you need it without too many dependencies.

I'm going to use this bank dataset from data.gov: https://catalog.data.gov/dataset/development-credit-authority-dca-data-set-loan-transactions-28508 renamed as `'credit_loan_dataset.csv'`. It's ~33MB of comma delimited CSV, containing 186,545 rows of raw data.

### Software & hardwares

This is based on miniconda IPython version 7.9.0 (Python 3.7.5) in macOS Mojave 10.14.6 with 16GB of 2133MHz memory chip and a cheap 120GB whitebranded **SATA** M.2 drive. YMMV on the actual performance on your machine, but regular SATA SSD should be comparable & PCIe NVMe M.2 is much faster.

In [2]:
import json, csv
from pprint import pprint

def open_csv(filename, dialect='excel'):
        
    with open(filename, 'r') as f:
        reader = csv.DictReader(f, dialect=dialect)
        for row in reader:
            yield row

In [2]:
# let's try and see if this works
# in python v.3.8 the output of `csv.DictWriter` is a `dict`, but v3 prior to 3.8 it's `OrderedDict`
data = open_csv('credit_loan_dataset.csv')
test = dict(next(data))

In [3]:
test

{'Guarantee Number': '099-DCA-09-006A (Asociacion Arariwa)',
 'Transaction Report ID': '356191',
 'Guarantee Country Name': 'Worldwide',
 'Amount (USD)': '980144.4043',
 'Currency Name': 'PERU - NUEVO SOL',
 'Disbursement Date': '09/20/2011 12:00:00 AM',
 'End Date': '09/09/2013 12:00:00 AM',
 'Business Sector': '',
 'City/Town': 'Cusco',
 'State/Province/Region Name': 'Cusco',
 'State/Province/Region Code': 'PE08',
 'State/Province/Region Country Name': 'Peru',
 'Region Name': 'LATIN AMERICA & THE CARIBBEAN',
 'Is Woman Owned?': '0',
 'Is First Time Borrower?': '1',
 'Business Size': '>100',
 'Latitude': '-13.518333',
 'Longitude': '-71.978056'}

In [4]:
# again
test

{'Guarantee Number': '099-DCA-09-006A (Asociacion Arariwa)',
 'Transaction Report ID': '356191',
 'Guarantee Country Name': 'Worldwide',
 'Amount (USD)': '980144.4043',
 'Currency Name': 'PERU - NUEVO SOL',
 'Disbursement Date': '09/20/2011 12:00:00 AM',
 'End Date': '09/09/2013 12:00:00 AM',
 'Business Sector': '',
 'City/Town': 'Cusco',
 'State/Province/Region Name': 'Cusco',
 'State/Province/Region Code': 'PE08',
 'State/Province/Region Country Name': 'Peru',
 'Region Name': 'LATIN AMERICA & THE CARIBBEAN',
 'Is Woman Owned?': '0',
 'Is First Time Borrower?': '1',
 'Business Size': '>100',
 'Latitude': '-13.518333',
 'Longitude': '-71.978056'}

In [8]:
%lsmagic # this is a list of ipython features that you can use in Jupyter notebooks

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

some useful links for `%magic`:

* https://ipython.readthedocs.io/en/stable/interactive/tutorial.html#magic-functions
* https://stackoverflow.com/questions/49136737/how-profiling-class-method-using-ipython-lprun-magic-function

Profilers are important, but benchmarking is a dark art, it's easy to get misleading or "wrong" result. To install external profiler on conda/miniconda use `conda install line_profiler`, or follow guide in https://github.com/rkern/line_profiler#installation. memory_profiler is also available on conda & can be installed with `pip install memory_profiler`

Make sure your in the right virtual env.


In [6]:
%load_ext line_profiler
%load_ext memory_profiler

In [5]:
import json, csv
from pprint import pprint

def open_csv(filename, rows, dialect='excel'):
        
    with open(filename, 'r') as f:
        reader = csv.DictReader(f, dialect=dialect)
        for count in range(rows):
            yield next(reader)


def get_csv_rows(filename, row_count):
    # there's no need to convert to dict when using python 3.8+: return list(get_csv_rows(...))
    csv_rows = [dict(d) for d in list(open_csv(filename, row_count))]
    return csv_rows


In [6]:
data = get_csv_rows('credit_loan_dataset.csv', 20)

In [7]:
len(data)

20

In [8]:
pprint({i: d['Transaction Report ID'] for i,d in enumerate(data)})

{0: '356191',
 1: '331620',
 2: '349835',
 3: '385807',
 4: '473058',
 5: '420674',
 6: '123514',
 7: '369568',
 8: '385756',
 9: '420663',
 10: '433368',
 11: '435375',
 12: '434741',
 13: '434767',
 14: '435376',
 15: '439987',
 16: '439991',
 17: '439992',
 18: '460017',
 19: '460019'}


In [179]:
%timeit get_csv_rows('credit_loan_dataset.csv', 20)

201 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [181]:
%lprun get_csv_rows('credit_loan_dataset.csv', 20)

In [190]:
%mprun -f get_csv_rows('credit_loan_dataset.csv', 20)

UsageError: Line magic function `%mprun` not found.


In [189]:
%memit?

Object `%memit` not found.


In [183]:
%%time
get_csv_rows('credit_loan_dataset.csv', 20)

CPU times: user 599 µs, sys: 546 µs, total: 1.15 ms
Wall time: 727 µs


[{'Guarantee Number': '099-DCA-09-006A (Asociacion Arariwa)',
  'Transaction Report ID': '356191',
  'Guarantee Country Name': 'Worldwide',
  'Amount (USD)': '980144.4043',
  'Currency Name': 'PERU - NUEVO SOL',
  'Disbursement Date': '09/20/2011 12:00:00 AM',
  'End Date': '09/09/2013 12:00:00 AM',
  'Business Sector': '',
  'City/Town': 'Cusco',
  'State/Province/Region Name': 'Cusco',
  'State/Province/Region Code': 'PE08',
  'State/Province/Region Country Name': 'Peru',
  'Region Name': 'LATIN AMERICA & THE CARIBBEAN',
  'Is Woman Owned?': '0',
  'Is First Time Borrower?': '1',
  'Business Size': '>100',
  'Latitude': '-13.518333',
  'Longitude': '-71.978056'},
 {'Guarantee Number': '099-DCA-09-006B (Pro Mujer Peru)',
  'Transaction Report ID': '331620',
  'Guarantee Country Name': 'Worldwide',
  'Amount (USD)': '1960288.809',
  'Currency Name': 'PERU - NUEVO SOL',
  'Disbursement Date': '09/20/2011 12:00:00 AM',
  'End Date': '09/04/2014 12:00:00 AM',
  'Business Sector': '',
  'C

In [184]:
%%time
get_csv_rows('credit_loan_dataset.csv', 500)

CPU times: user 5.07 ms, sys: 331 µs, total: 5.4 ms
Wall time: 5.15 ms


[{'Guarantee Number': '099-DCA-09-006A (Asociacion Arariwa)',
  'Transaction Report ID': '356191',
  'Guarantee Country Name': 'Worldwide',
  'Amount (USD)': '980144.4043',
  'Currency Name': 'PERU - NUEVO SOL',
  'Disbursement Date': '09/20/2011 12:00:00 AM',
  'End Date': '09/09/2013 12:00:00 AM',
  'Business Sector': '',
  'City/Town': 'Cusco',
  'State/Province/Region Name': 'Cusco',
  'State/Province/Region Code': 'PE08',
  'State/Province/Region Country Name': 'Peru',
  'Region Name': 'LATIN AMERICA & THE CARIBBEAN',
  'Is Woman Owned?': '0',
  'Is First Time Borrower?': '1',
  'Business Size': '>100',
  'Latitude': '-13.518333',
  'Longitude': '-71.978056'},
 {'Guarantee Number': '099-DCA-09-006B (Pro Mujer Peru)',
  'Transaction Report ID': '331620',
  'Guarantee Country Name': 'Worldwide',
  'Amount (USD)': '1960288.809',
  'Currency Name': 'PERU - NUEVO SOL',
  'Disbursement Date': '09/20/2011 12:00:00 AM',
  'End Date': '09/04/2014 12:00:00 AM',
  'Business Sector': '',
  'C

In [187]:
!python -m cProfile -s cumtime chunk_csv.py

[{'Amount (USD)': '980144.4043',
  'Business Sector': '',
  'Business Size': '>100',
  'City/Town': 'Cusco',
  'Currency Name': 'PERU - NUEVO SOL',
  'Disbursement Date': '09/20/2011 12:00:00 AM',
  'End Date': '09/09/2013 12:00:00 AM',
  'Guarantee Country Name': 'Worldwide',
  'Guarantee Number': '099-DCA-09-006A (Asociacion Arariwa)',
  'Is First Time Borrower?': '1',
  'Is Woman Owned?': '0',
  'Latitude': '-13.518333',
  'Longitude': '-71.978056',
  'Region Name': 'LATIN AMERICA & THE CARIBBEAN',
  'State/Province/Region Code': 'PE08',
  'State/Province/Region Country Name': 'Peru',
  'State/Province/Region Name': 'Cusco',
  'Transaction Report ID': '356191'},
 {'Amount (USD)': '1960288.809',
  'Business Sector': '',
  'Business Size': '>100',
  'City/Town': 'Puno',
  'Currency Name': 'PERU - NUEVO SOL',
  'Disbursement Date': '09/20/2011 12:00:00 AM',
  'End Date': '09/04/2014 12:00:00 AM',
  'Guarantee Country Name': 'Worldwide',
  'Guarantee Number': '099-DCA-09-006B (Pro Mujer

 {'Amount (USD)': '94073.3772',
  'Business Sector': 'Trade/Commerce',
  'Business Size': '1--5',
  'City/Town': 'Rivonia',
  'Currency Name': 'SOUTH AFRICA - RAND',
  'Disbursement Date': '10/14/2013 12:00:00 AM',
  'End Date': '09/14/2018 12:00:00 AM',
  'Guarantee Country Name': 'South Africa',
  'Guarantee Number': '674-DCA-11-016',
  'Is First Time Borrower?': '1',
  'Is Woman Owned?': '0',
  'Latitude': '-26.053154',
  'Longitude': '28.059594',
  'Region Name': 'AFRICA',
  'State/Province/Region Code': 'SF06',
  'State/Province/Region Country Name': 'South Africa',
  'State/Province/Region Name': 'Gauteng',
  'Transaction Report ID': '439987'},
 {'Amount (USD)': '282220.1317',
  'Business Sector': 'Trade/Commerce',
  'Business Size': '11--50',
  'City/Town': 'Sanbel',
  'Currency Name': 'SOUTH AFRICA - RAND',
  'Disbursement Date': '10/28/2013 12:00:00 AM',
  'End Date': '09/28/2018 12:00:00 AM',
  'Guarantee Country Name': 'South Africa',
  'Guarantee Number': '674-DCA-11-016',
